In [3]:
import os
import random
import sys
sys.path.append('../')
from sonicrl.worldmodel.autoencoder import autoencoder, load_image

import cv2
import numpy as np
import ipywidgets as widgets

from IPython.display import display

/home/ben/virtualenvs/sonicrl/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [10]:
ENCODER_WEIGHTS = '/home/ben/PycharmProjects/sonicrl/checkpoints/encoder.01-33845.851172.hdf5'
DECODER_WEIGHTS = '/home/ben/PycharmProjects/sonicrl/checkpoints/decoder.01-33845.851172.hdf5'

FRAMES_DIRECTORY = '/home/ben/PycharmProjects/frames/'

In [5]:
def get_image_bytes(path):
    with open(path, 'rb') as f:
        return f.read()

def encode_image(image):
    image = image * 255.
    image = image.astype(np.uint8)
    _, image_bytes = cv2.imencode('.jpg', image)
    return image_bytes.tobytes()

In [6]:
_, encoder, decoder = autoencoder((224, 320, 3))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 320, 3)       0         
_________________________________________________________________
encoder (Model)              [(None, 64), (None, 64),  36792532  
_________________________________________________________________
decoder (Model)              (None, 224, 320, 3)       37107715  
Total params: 73,900,247
Trainable params: 73,900,247
Non-trainable params: 0
_________________________________________________________________


../sonicrl/worldmodel/autoencoder.py:111: UserWarning: Output "decoder" missing from loss dictionary. We assume this was done on purpose, and we will not be expecting any data to be passed to "decoder" during training.
  vae.compile(optimizer='rmsprop')


In [11]:
encoder.load_weights(ENCODER_WEIGHTS)
decoder.load_weights(DECODER_WEIGHTS)

In [8]:
paths = [os.path.join(FRAMES_DIRECTORY, fn) for fn in os.listdir(FRAMES_DIRECTORY)]

In [9]:
def autoencode_random_image(_):
    path = random.choice(paths)
    image_bytes = get_image_bytes(path)
    source_widget.value = image_bytes
    image = load_image(path)
    
    z_mean, _, _ = encoder.predict(np.expand_dims(image, 0))
    
    reconstructed = decoder.predict(z_mean)[0]
    reconstructed_bytes = encode_image(reconstructed)
    reconstructed_widget.value = reconstructed_bytes
    
trigger = widgets.Button(description='Random Image')
trigger.on_click(autoencode_random_image)

source_widget = widgets.Image(format='jpg', height=224, width=320)
reconstructed_widget = widgets.Image(format='jpg', height=224, width=320)

image_layout = widgets.HBox([source_widget, reconstructed_widget])
layout = widgets.VBox([trigger, image_layout])
display(layout)
autoencode_random_image(None)